In [1]:
import torch
import torch.utils.data as Data
import torch.nn.functional as F
from torch.autograd import Variable
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
torch.manual_seed(1)    # reproducible

# Generate some fake data

In [3]:
# fake dataset
x = torch.unsqueeze(torch.linspace(-1, 1, 1000), dim=1)
y = x.pow(2) + 0.1*torch.normal(torch.zeros(*x.size()))

# plot dataset
plt.scatter(x.numpy(), y.numpy())
plt.show()

# Load dataset into a torch dataset

In [6]:
LR = 0.01
BATCH_SIZE = 32
EPOCH = 12

In [7]:
torch_dataset = Data.TensorDataset(data_tensor=x, target_tensor=y)
loader = Data.DataLoader(
    dataset=torch_dataset, 
    batch_size=BATCH_SIZE, 
    shuffle=True, num_workers=2,)

# Default Model

In [8]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.hidden = torch.nn.Linear(1, 20)
        self.predict = torch.nn.Linear(20, 1)
        
    def forward(self, x):
        x = F.relu(self.hidden(x))
        x = self.predict(x)
        return x

# Different Networks

In [12]:
net_SGD         = Net()
net_Momentum    = Net()
net_RMSprop     = Net()
net_Adam        = Net()
net_Adadelta    = Net()
nets = [net_SGD, net_Momentum, net_RMSprop, net_Adam, net_Adadelta]

# Different Optimizers

In [13]:
opt_SGD         = torch.optim.SGD(net_SGD.parameters(), lr=LR)
opt_Momentum    = torch.optim.SGD(net_Momentum.parameters(), lr=LR, momentum=0.8)
opt_RMSprop     = torch.optim.RMSprop(net_RMSprop.parameters(), lr=LR, alpha=0.9)
opt_Adam        = torch.optim.Adam(net_Adam.parameters(), lr=LR, betas=(0.9, 0.99))
opt_Adadelta    = torch.optim.Adadelta(net_Adadelta.parameters(), lr=LR, rho=0.9, eps=1e-04, weight_decay=0.001)
optimizers = [opt_SGD, opt_Momentum, opt_RMSprop, opt_Adam, opt_Adadelta]

In [14]:
loss_func = torch.nn.MSELoss()
losses_his = [[], [], [], []]   # record loss

# Training

In [ ]:
for epoch in range(EPOCH):
    print('Epoch: ', epoch)
    for step, (batch_x, batch_y) in enumerate(loader):
        b_x = Variable(batch_x)
        b_y = Variable(batch_y)
        
        for net, opt, l_his in zip(nets, optimizers, losses_his):
            output = net(b_x)                 # Get output for every net
            loss = loss_func(output, b_y)     # compute loss for every net
            opt.zero_grad()                   # Clear gradients for next train
            loss.backward()                   # backpropagation, compute gradients 